In [1337]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


In [1338]:
#reading csv file
gas_df = pd.read_csv('Nat_Gas.csv')
gas_df.head()

gas_df.dropna()

,Dates,Prices
0,10/31/20,10.10
1,11/30/20,10.30
2,12/31/20,11.00
3,1/31/21,10.90
4,2/28/21,10.90
5,3/31/21,10.90
6,4/30/21,10.40
7,5/31/21,9.84
8,6/30/21,10.00
9,7/31/21,10.10


In [1339]:
##feature engineering

#Splitting the Date column into days, months and year
# Convert the 'Dates' column to datetime format
gas_df['Dates'] = pd.to_datetime(gas_df['Dates'], format='%m/%d/%y')

gas_df['Year'] = gas_df['Dates'].dt.year
gas_df['Month'] = gas_df['Dates'].dt.month


gas_df.head()

,Dates,Prices,Year,Month
0,2020-10-31,10.1,2020,10
1,2020-11-30,10.3,2020,11
2,2020-12-31,11.0,2020,12
3,2021-01-31,10.9,2021,1
4,2021-02-28,10.9,2021,2


In [1340]:
#checking for null values 
gas_df.duplicated().sum().any()

False

In [1341]:
gas_df.isnull().values.any()

False

In [1342]:
#price list created below from task 1 , slight modifications to avoid 'date' value erros
def next_year_price(date):
    date = pd.to_datetime(date)
    prices_next_year = []
    if date in gas_df['Dates'].values:
        return gas_df.loc[gas_df['Dates'] == date, 'Prices'].values[0]
    for i in np.arange(12):
        X = np.array(gas_df[gas_df['Month'] == 1+i]['Year']).reshape(-1,1)
      
        y = np.array(gas_df[gas_df['Month'] == 1+ i]['Prices'])
        regress = LinearRegression().fit(X,y)
        predictions = regress.predict([[date]])
        prices_next_year.append(round(float(predictions), 2))

    return prices_next_year


In [1343]:
#CREATING A PRICING GAS CONTRACT MODEL

def contract_value(injection_dates, withdrawal_dates, inject_rates, withdrawal_costs, maximum_volume, storage_cost):

    
    for i in range (len(injection_dates)):
        for j in range(len(withdrawal_dates)):
        
                
            injection_price = next_year_price(pd.to_datetime(injection_dates[i]))

            withdrawal_price = next_year_price(pd.to_datetime(withdrawal_dates[j]))

          

            storage_time_months = round((pd.to_datetime(withdrawal_dates[j]) - pd.to_datetime(injection_dates[i])).days)*12/365

            total_volume = min(inject_rates*storage_time_months, storage_time_months)*100000

            total_storage_cost = storage_cost * storage_time_months

            injection_cost = total_volume*injection_price - (total_volume/100000*withdrawal_costs)

            revenue = total_volume*withdrawal_price - (total_volume/100000*withdrawal_costs)

            total_transportation_costs = transportation_costs*2

            total_value = +-(revenue - injection_cost - total_storage_cost - total_transportation_costs)


        # printing the results

            print('Injection Price' , injection_price)
            print('Withdrawal Price', withdrawal_price)
            print('storage time in months', storage_time_months)
            print('Total Volume', total_volume)
            print('Total Storage Costs', total_storage_cost)
            print('Injection Costs', injection_cost)
            print('Total Transportation Costs', total_transportation_costs)
            print('Total Revenue', revenue)
        #    print('Total Value', total_value)
        

    

    return total_value
        

In [1344]:
#TESTING THE FORMULA ABOVE WITH SAMPLE VALUES
injection_dates = ['10/31/2022']
withdrawal_dates = ['07/31/2023']
inject_rates = 1000000
withdrawal_costs = 10000
maximum_volume = 50000000
storage_cost = 100000
transportation_costs = 50000

In [1345]:
#FUNCTION TO GIVE THE VALUE OF THE CONTRACT

print ('CONTRACT VALUE', contract_value(injection_dates, withdrawal_dates, inject_rates, withdrawal_costs, maximum_volume, storage_cost))

Injection Price 11.0
Withdrawal Price 11.4
storage time in months 8.975342465753425
Total Volume 897534.2465753425
Total Storage Costs 897534.2465753425
Injection Costs 9783123.287671233
Total Transportation Costs 100000
Total Revenue 10142136.98630137
CONTRACT VALUE 638520.5479452051
